In [1]:
#Ref: https://github.com/VBhat97/Gale-Shapley-Algorithm/blob/master/Gale_Shapely_Algorithm.ipynb
import random

from typing import List, Dict, Tuple

# Declaring types
Person = str
People = List[Person]
Preferences = List[Person]
Side = Dict[Person, Preferences]
Participants = Dict[str, Side]
Pair = Tuple[Person, Person]
Matching = Dict[Person, Person]
Stable_Matching = List[Pair]


class MissingPreferences(Exception):
    pass

def other_side(current_side: str, all_sides: list) -> str:
    """Given a side and all possible sides, returns the opposite side"""
    for s in all_sides:
        if s != current_side:
            return s
     
def all_preferences(participants: Participants) -> bool:
    """Checks whether all participants have all participants of the other side in their own preference"""
    sides = list(participants.keys())
    for side, people in participants.items():
        other_side_participants = participants[other_side(side, sides)]
        for name, preferences in people.items():
            for o in other_side_participants:
                if o not in preferences:
                    return False
    else:
        return True
 
def is_free(person: Person, engaged: Matching) -> bool:
    """Is the person missing from all current pairs?"""
    return person not in engaged

def current_match(person: Person, engaged: Matching) -> bool:
    """Returns the current match for that person"""
    return engaged.get(person)

def free_participants(people: People, engaged: Matching) -> list:
    """Returns all participants are that are still currently free"""
    return filter(lambda x: x not in engaged, people)

def preferred(a: Person, b: Person, preferences: Preferences) -> Person:
    """Is a preferred over b according to the preferences ordering?"""
    for preference in preferences:
        if preference == a:
            return True
        if preference == b:
            return False

def stable_matching(participants: Participants) -> Stable_Matching:
    """For a group of participants and their respective preferences of the other group, returns a list of stable matches according to the Gale–Shapley algorithm"""
    # The algorithm requires each participant expresses a preference that includes all other participants
    if not all_preferences(participants):
        raise MissingPreferences

    sides = list(participants.keys())
    proposing = sides[0]  # Taking the 1st side
    receiving = sides[1]  # Taking the 2nd side
    proposers = participants[proposing]
    receivers = participants[receiving]
    free_proposers = proposers
    proposal_history = {k: {} for k in proposers}
    engagements = {}

    while free_proposers:
        for proposer in free_proposers:
            preferences = proposers[proposer]
            for target in preferences:
                # Has proposed yet?
                if target not in proposal_history[proposer]:
                    # Record proposal
                    proposal_history[proposer][target] = ""
                    # Is receiver free?
                    if is_free(target, engagements):
                        # Engagement
                        engagements[proposer] = target
                        engagements[target] = proposer
                    else:
                        # Pair already exists
                        current = current_match(target, engagements)
                        target_preferences = receivers[target]
                        if preferred(proposer, current, target_preferences):
                            # Proposer replaces the current individual
                            engagements[target] = proposer
                            engagements[proposer] = target
                            # Freeing the incumbent
                            del engagements[current]
                    # Done proposing this round
                    break
        # Updating the list of proposers that are free
        # Must be a list since a generator always evaluates to True
        free_proposers = list(free_participants(free_proposers, engagements))

    # Composing the stable matchings
    stable_matchings = set()
    for a, b in engagements.items():
        # Checking the reverse isn't already in
        if (b, a) not in stable_matchings:
            stable_matchings.add((a, b))


    return list(stable_matchings)

if __name__ == "__main__":
    listA = ["A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8"]
    listB = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8"]
    results = []
    count = 0
    for i in range(1000):        
        sample_participants = {
            "Super_Group_A": {
                "A1": random.sample(listB, 8),
                "A2": random.sample(listB, 8),
                "A3": random.sample(listB, 8),
                "A4": random.sample(listB, 8),
                "A5": random.sample(listB, 8),
                "A6": random.sample(listB, 8),
                "A7": random.sample(listB, 8),
                "A8": random.sample(listB, 8)
            },
            "Super_Group_B": {
                "B1": random.sample(listA, 8),
                "B2": random.sample(listA, 8),
                "B3": random.sample(listA, 8),
                "B4": random.sample(listA, 8),
                "B5": random.sample(listA, 8),
                "B6": random.sample(listA, 8),
                "B7": random.sample(listA, 8),
                "B8": random.sample(listA, 8)
            }
        }
        result = stable_matching(sample_participants)
        results.append(result)

    for result in results:
        if result is not results[-1]:
            count += 1
    p = count/1000
    print(p)
        

0.999
